# Data Science, Fall 2024


[Acknowledgments Page](https://ds100.org/fa23/acks/)

A demo of data cleaning and exploratory data analysis using the CDC Tuberculosis data and the Mauna Loa CO2 data.

In [1]:
import numpy as np
import pandas as pd
import json

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
#%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 9)

sns.set()
sns.set_context('talk')
np.set_printoptions(threshold=20, precision=2, suppress=True)
pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)
# This option stops scientific notation for pandas
pd.set_option('display.float_format', '{:.2f}'.format)

# Silence some spurious seaborn warnings
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

## Structure: Different File Formats

There are many file types for storing structured data: CSV, TSV, JSON, XML, ASCII, SAS...
* Documentation will be your best friend to understand how to process many of these file types.
* In lecture, we will cover TSV and JSON since pandas supports them out-of-box.

### TSV

**TSV** (Tab-Separated Values) files are very similar to CSVs, but now items are delimited by tabs.

The `pd.read_csv` function also reads in TSVs if we specify the **delimiter** with parameter `sep='\t'` ([documentation](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html)).

In [5]:
# Read TSV file
data_json = pd.read_csv("cdc_tuberculosis.tsv",delimiter="\t")
data_json

,Unnamed: 0,No. of TB cases,Unnamed: 2,Unnamed: 3,TB incidence,Unnamed: 5,Unnamed: 6
0,U.S. jurisdiction,2019,2020,2021,2019.00,2020.00,2021.00
1,Total,"8,900","7,173","7,860",2.71,2.16,2.37
2,Alabama,87,72,92,1.77,1.43,1.83
3,Alaska,58,58,58,7.91,7.92,7.92
4,Arizona,183,136,129,2.51,1.89,1.77
...,...,...,...,...,...,...,...
48,Virginia,191,169,161,2.23,1.96,1.86
49,Washington,221,163,199,2.90,2.11,2.57
50,West Virginia,9,13,7,0.50,0.73,0.39
51,Wisconsin,51,35,66,0.88,0.59,1.12


*Side note*: there was a question last time on how pandas differentiates a comma delimiter vs. a comma within the field itself, e.g., `8,900`. Check out the documentation for the `quotechar` parameter.

### JSON
The City of Berkeley Open Data [website](https://data.cityofberkeley.info/Health/COVID-19-Confirmed-Cases/xn6j-b766) has a dataset with COVID-19 Confirmed Cases among Berkeley residents by date.


We can download this file, saving it as a JSON (note the source URL file type).

In the interest of **reproducible data science** we will download the data programatically.  We have defined some helper functions in the below.  I can then reuse this helper functions in many different notebooks.

In [6]:
import requests
from pathlib import Path
import time
def fetch_and_cache(data_url, file, data_dir="data", force=False):
    """
    Download and cache a url and return the file object.

    data_url: the web address to download
    file: the file in which to save the results.
    data_dir: (default="data") the location to save the data
    force: if true the file is always re-downloaded

    return: The pathlib.Path object representing the file.
    """

    ### BEGIN SOLUTION
    data_dir = Path(data_dir)
    data_dir.mkdir(exist_ok = True)
    file_path = data_dir / Path(file)
    # If the file already exists and we want to force a download then
    # delete the file first so that the creation date is correct.
    if force and file_path.exists():
        file_path.unlink()
    if force or not file_path.exists():
        print('Downloading...', end=' ')
        resp = requests.get(data_url)
        with file_path.open('wb') as f:
            f.write(resp.content)
        print('Done!')
        last_modified_time = time.ctime(file_path.stat().st_mtime)
    else:
        last_modified_time = time.ctime(file_path.stat().st_mtime)
        print("Using cached version that was downloaded (UTC):", last_modified_time)
    return file_path
    ### END SOLUTION


In [7]:
'''covid_file = fetch_and_cache(
    "https://data.cityofberkeley.info/api/views/xn6j-b766/rows.json?accessType=DOWNLOAD",
    "confirmed-cases.json",
    force=False)
covid_file  '''        # a file path wrapper object

'covid_file = fetch_and_cache(\n    "https://data.cityofberkeley.info/api/views/xn6j-b766/rows.json?accessType=DOWNLOAD",\n    "confirmed-cases.json",\n    force=False)\ncovid_file  '

#### File size

Often, I like to start my analysis by getting a rough estimate of the size of the data.  This will help inform the tools I use and how I view the data.  If it is relatively small I might use a text editor or a spreadsheet to look at the data.  If it is larger, I might jump to more programmatic exploration or even used distributed computing tools.

However here we will use Python tools to probe the file.

Since these seem to be text files I might also want to investigate the number of lines, which often corresponds to the number of records.

### EDA: Digging into JSON

Python has relatively good support for JSON data since it closely matches the internal python object model.  In the following cell we import the entire JSON datafile into a python dictionary using the `json` package.

In [8]:
# load JSON file
covid_file="confirmed-cases.json"

# get size of file using getsize function
import os

# Total Count of lines
with open(covid_file, "r") as f:
    covid_json = json.load(f)


In [9]:
import os

print(covid_file , "is", os.path.getsize(covid_file))
file_size = os.path.getsize(covid_file)
file_size
print(covid_file,"is",file_size/1e6,"MB")

confirmed-cases.json is 265382
confirmed-cases.json is 0.265382 MB


The `covid_json` variable is now a dictionary encoding the data in the file:

In [52]:
# find type of your file
type(covid_json)

dict

#### Examine what keys are in the top level json object

We can list the keys to determine what data is stored in the object.

In [53]:
# identify keys() of JSON file

covid_json.keys()

dict_keys(['meta', 'data'])

**Observation**: The JSON dictionary contains a `meta` key which likely refers to meta data (data about the data).  Meta data often maintained with the data and can be a good source of additional information.

<br/>

We can investigate the meta data further by examining the keys associated with the metadata.

In [57]:
# Further explore meta key()
covid_json['meta'].keys()

dict_keys(['view'])

The `meta` key contains another dictionary called `view`.  This likely refers to meta-data about a particular "view" of some underlying database.  We will learn more about views when we study SQL later in the class.    

In [10]:
# Further explore view key()
covid_json['meta']['view'].keys()

dict_keys(['id', 'name', 'assetType', 'attribution', 'averageRating', 'category', 'createdAt', 'description', 'displayType', 'downloadCount', 'hideFromCatalog', 'hideFromDataJson', 'locked', 'newBackend', 'numberOfComments', 'oid', 'provenance', 'publicationAppendEnabled', 'publicationDate', 'publicationGroup', 'publicationStage', 'rowsUpdatedAt', 'rowsUpdatedBy', 'tableId', 'totalTimesRated', 'viewCount', 'viewLastModified', 'viewType', 'approvals', 'clientContext', 'columns', 'grants', 'metadata', 'owner', 'query', 'rights', 'tableAuthor', 'tags', 'flags'])

In [11]:
covid_json['meta']['view']['columns']

[{'id': -1,
  'name': 'sid',
  'dataTypeName': 'meta_data',
  'fieldName': ':sid',
  'position': 0,
  'renderTypeName': 'meta_data',
  'format': {},
  'flags': ['hidden']},
 {'id': -1,
  'name': 'id',
  'dataTypeName': 'meta_data',
  'fieldName': ':id',
  'position': 0,
  'renderTypeName': 'meta_data',
  'format': {},
  'flags': ['hidden']},
 {'id': -1,
  'name': 'position',
  'dataTypeName': 'meta_data',
  'fieldName': ':position',
  'position': 0,
  'renderTypeName': 'meta_data',
  'format': {},
  'flags': ['hidden']},
 {'id': -1,
  'name': 'created_at',
  'dataTypeName': 'meta_data',
  'fieldName': ':created_at',
  'position': 0,
  'renderTypeName': 'meta_data',
  'format': {},
  'flags': ['hidden']},
 {'id': -1,
  'name': 'created_meta',
  'dataTypeName': 'meta_data',
  'fieldName': ':created_meta',
  'position': 0,
  'renderTypeName': 'meta_data',
  'format': {},
  'flags': ['hidden']},
 {'id': -1,
  'name': 'updated_at',
  'dataTypeName': 'meta_data',
  'fieldName': ':updated_at'

In [12]:
covid_json['meta']['view']['columns'][0]['name']
    

'sid'

In [14]:
rows=covid_json["data"]
fields = [column['name'] for column in covid_json['meta']['view']['columns']]
data=pd.DataFrame(rows,columns = fields)
data

,sid,id,position,created_at,created_meta,updated_at,updated_meta,meta,Date,New Cases,Cumulative Cases
0,row-rb9f.5nek-8ine,00000000-0000-0000-4662-FC7183571076,0,1721778125,None,1721778125,None,{ },2019-12-01T00:00:00,0,0
1,row-vztw-e5xz~k7e6,00000000-0000-0000-4572-4B7999484114,0,1721778125,None,1721778125,None,{ },2019-12-02T00:00:00,0,0
2,row-bjvt.2dfe.85rq,00000000-0000-0000-7CFB-15321FF8BFED,0,1721778125,None,1721778125,None,{ },2019-12-03T00:00:00,0,0
3,row-u3xy-3xcz_xc3n,00000000-0000-0000-8EB1-2012B4B4B530,0,1721778125,None,1721778125,None,{ },2019-12-04T00:00:00,0,0
4,row-na7q-wzeq.h924,00000000-0000-0000-2FD9-335951D74781,0,1721778125,None,1721778125,None,{ },2019-12-05T00:00:00,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1690,row-d28u_ew2y~ms8f,00000000-0000-0000-7C6A-161A637B9C10,0,1721778125,None,1721778125,None,{ },2024-07-17T00:00:00,6,25198
1691,row-nmtf.ke2i~yb3k,00000000-0000-0000-04BA-221AC1453106,0,1721778125,None,1721778125,None,{ },2024-07-18T00:00:00,5,25203
1692,row-4y8w-m68e~d3t5,00000000-0000-0000-F211-E2C7B20961B2,0,1721778125,None,1721778125,None,{ },2024-07-19T00:00:00,2,25205
1693,row-riyz.7u5e.ivns,00000000-0000-0000-20B4-50F4576EBBF2,0,1721778125,None,1721778125,None,{ },2024-07-20T00:00:00,1,25206


In [15]:
for column in covid_json['meta']['view']['columns']:
    print(column['name'])

sid
id
position
created_at
created_meta
updated_at
updated_meta
meta
Date
New Cases
Cumulative Cases


In [16]:
covid_json['meta']['view']['columns'][10]['name']

'Cumulative Cases'

Notice that this a nested/recursive data structure.  As we dig deeper we reveal more and more keys and the corresponding data:

```
meta
|-> data
    | ... (haven't explored yet)
|-> view
    | -> id
    | -> name
    | -> attribution
    ...
    | -> description
    ...
    | -> columns
    ...
```

There is a key called description in the view sub dictionary.  This likely contains a description of the data:

In [12]:
# use description key from view dictionary

Counts of confirmed COVID-19 cases among Berkeley residents by date. As of 6/21/22, this dataset will be updated weekly instead of daily. As of 11/14/22, this dataset only includes PCR cases.



#### Examining the Data Field for Records

We can look at a few entries in the `data` field. This is what we'll load into Pandas.


In [18]:
# explore data key and print some entries


0|['row-rb9f.5nek-8ine', '00000000-0000-0000-4662-FC7183571076', 0, 1721778125, None, 1721778125, None, '{ }', '2019-12-01T00:00:00', '0', '0']
1|['row-vztw-e5xz~k7e6', '00000000-0000-0000-4572-4B7999484114', 0, 1721778125, None, 1721778125, None, '{ }', '2019-12-02T00:00:00', '0', '0']
2|['row-bjvt.2dfe.85rq', '00000000-0000-0000-7CFB-15321FF8BFED', 0, 1721778125, None, 1721778125, None, '{ }', '2019-12-03T00:00:00', '0', '0']


Observations:
* These look like equal-length records, so maybe `data` is a table!
* But what do each of values in the record mean? Where can we find column headers?

Back to the metadata.

#### Columns Metadata

Another potentially useful key in the metadata dictionary is the `columns`.  This returns a list:

In [14]:
# check type of columns key


list

Let's go back to the file explorer.

Based on the contents of this key, what are reasonable names for each column in the `data` table?

#### Summary of exploring the JSON file

1. The above **metadata** tells us a lot about the columns in the data including column names, potential data anomalies, and a basic statistic.
1. Because of its non-tabular structure, JSON makes it easier (than CSV) to create **self-documenting data**, meaning that information about the data is stored in the same file as the data.
1. Self documenting data can be helpful since it maintains its own description and these descriptions are more likely to be updated as data changes.

### JSON with pandas

After our above EDA, let's finally go about loading the data (not the metadata) into a pandas dataframe.

In the following block of code we:
1. Translate the JSON records into a dataframe:

    * fields: `covid_json['meta']['view']['columns']`
    * records: `covid_json['data']`
    
1. Remove columns that have no metadata description.  This would be a bad idea in general but here we remove these columns since the above analysis suggests that they are unlikely to contain useful information.
1. Examine the `tail` of the table.

In [ ]:
# Load the data from JSON and assign column titles


,sid,id,position,created_at,created_meta,updated_at,updated_meta,meta,Date,New Cases,Cumulative Cases
1292,row-zg3v~6bpx~tp8j,00000000-0000-0000-7710-AECCCEB67A5A,0,1687370769,None,1687370769,None,{ },2023-06-15T00:00:00,2,23384
1293,row-wean-73e3.fixn,00000000-0000-0000-27C0-D21340C3F93F,0,1687370769,None,1687370769,None,{ },2023-06-16T00:00:00,4,23388
1294,row-cvez.6nyd.3f6y,00000000-0000-0000-C1AE-C76310119F30,0,1687370769,None,1687370769,None,{ },2023-06-17T00:00:00,2,23390
1295,row-hzcg-24ra_7ipd,00000000-0000-0000-D048-D04E71C55E3A,0,1687370769,None,1687370769,None,{ },2023-06-18T00:00:00,0,23390
1296,row-8tdn_tuw8_hujn,00000000-0000-0000-3181-E8EECFBF2249,0,1687370769,None,1687370769,None,{ },2023-06-19T00:00:00,0,23390


<br/>

---


## Temporality

Let's briefly look at how we can use pandas `dt` accessors to work with dates/times in a dataset.

We will use the dataset from Lab 3: the Berkeley PD Calls for Service dataset.

In [17]:
# use data/Berkeley_PD_-_Calls_for_Service.csv now

calls=pd.read_csv("Berkeley_PD_-_Calls_for_Service.csv")
calls

,CASENO,OFFENSE,EVENTDT,EVENTTM,CVLEGEND,CVDOW,InDbDate,Block_Location,BLKADDR,City,State
0,18022300,DISTURBANCE,04/18/2018 12:00:00 AM,22:17,DISORDERLY CONDUCT,3,09/06/2018 03:30:12 AM,"OREGON STREET &amp; MCGEE AVE\nBerkeley, CA\n(...",OREGON STREET & MCGEE AVE,Berkeley,CA
1,18026683,THEFT MISD. (UNDER $950),05/09/2018 12:00:00 AM,21:25,LARCENY,3,09/06/2018 03:30:13 AM,"200 UNIVERSITY AVE\nBerkeley, CA\n(37.865511, ...",200 UNIVERSITY AVE,Berkeley,CA
2,18038550,THEFT MISD. (UNDER $950),05/18/2018 12:00:00 AM,20:00,LARCENY,5,09/06/2018 03:30:09 AM,"2200 MILVIA ST\nBerkeley, CA\n(37.868574, -122...",2200 MILVIA ST,Berkeley,CA
3,18014810,BURGLARY AUTO,03/13/2018 12:00:00 AM,08:50,BURGLARY - VEHICLE,2,09/06/2018 03:30:08 AM,"1200 SIXTH ST\nBerkeley, CA\n(37.881142, -122....",1200 SIXTH ST,Berkeley,CA
4,18018643,ALCOHOL OFFENSE,03/31/2018 12:00:00 AM,13:29,LIQUOR LAW VIOLATION,6,09/06/2018 03:30:11 AM,"CENTER STREET &amp; SHATTUCK AVE\nBerkeley, CA...",CENTER STREET & SHATTUCK AVE,Berkeley,CA
...,...,...,...,...,...,...,...,...,...,...,...
3783,18045829,THEFT MISD. (UNDER $950),08/15/2018 12:00:00 AM,08:42,LARCENY,3,09/06/2018 03:30:10 AM,"2300 TELEGRAPH AVE\nBerkeley, CA\n(37.868714, ...",2300 TELEGRAPH AVE,Berkeley,CA
3784,18040137,DISTURBANCE,07/17/2018 12:00:00 AM,10:34,DISORDERLY CONDUCT,2,09/06/2018 03:30:13 AM,"1100 UNIVERSITY AVE\nBerkeley, CA\n(37.869067,...",1100 UNIVERSITY AVE,Berkeley,CA
3785,18090816,VANDALISM,05/16/2018 12:00:00 AM,20:00,VANDALISM,3,09/06/2018 03:30:13 AM,"800 VICENTE RD\nBerkeley, CA\n",800 VICENTE RD,Berkeley,CA
3786,18024397,SEXUAL ASSAULT FEL.,04/28/2018 12:00:00 AM,17:00,SEX CRIME,6,09/06/2018 03:30:12 AM,"2700 BANCROFT WAY\nBerkeley, CA\n(37.869312, -...",2700 BANCROFT WAY,Berkeley,CA


Looks like there are three columns with dates/times: `EVENTDT`, `EVENTTM`, and `InDbDate`.

Most likely, `EVENTDT` stands for the date when the event took place, `EVENTTM` stands for the time of day the event took place (in 24-hr format), and `InDbDate` is the date this call is recorded onto the database.

If we check the data type of these columns, we will see they are stored as strings. We can convert them to `datetime` objects using pandas `to_datetime` function.

In [18]:
# pd.to_datetime on EVENTDT
calls["EVENTDT"]=pd.to_datetime(calls["EVENTDT"] )
calls["EVENTTM"]=pd.to_datetime(calls["EVENTTM"]).dt.time

calls

<ipython-input-18-d3aefc784c61>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  calls["EVENTDT"]=pd.to_datetime(calls["EVENTDT"] )
<ipython-input-18-d3aefc784c61>:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  calls["EVENTTM"]=pd.to_datetime(calls["EVENTTM"]).dt.time


,CASENO,OFFENSE,EVENTDT,EVENTTM,CVLEGEND,CVDOW,InDbDate,Block_Location,BLKADDR,City,State
0,18022300,DISTURBANCE,2018-04-18,22:17:00,DISORDERLY CONDUCT,3,09/06/2018 03:30:12 AM,"OREGON STREET &amp; MCGEE AVE\nBerkeley, CA\n(...",OREGON STREET & MCGEE AVE,Berkeley,CA
1,18026683,THEFT MISD. (UNDER $950),2018-05-09,21:25:00,LARCENY,3,09/06/2018 03:30:13 AM,"200 UNIVERSITY AVE\nBerkeley, CA\n(37.865511, ...",200 UNIVERSITY AVE,Berkeley,CA
2,18038550,THEFT MISD. (UNDER $950),2018-05-18,20:00:00,LARCENY,5,09/06/2018 03:30:09 AM,"2200 MILVIA ST\nBerkeley, CA\n(37.868574, -122...",2200 MILVIA ST,Berkeley,CA
3,18014810,BURGLARY AUTO,2018-03-13,08:50:00,BURGLARY - VEHICLE,2,09/06/2018 03:30:08 AM,"1200 SIXTH ST\nBerkeley, CA\n(37.881142, -122....",1200 SIXTH ST,Berkeley,CA
4,18018643,ALCOHOL OFFENSE,2018-03-31,13:29:00,LIQUOR LAW VIOLATION,6,09/06/2018 03:30:11 AM,"CENTER STREET &amp; SHATTUCK AVE\nBerkeley, CA...",CENTER STREET & SHATTUCK AVE,Berkeley,CA
...,...,...,...,...,...,...,...,...,...,...,...
3783,18045829,THEFT MISD. (UNDER $950),2018-08-15,08:42:00,LARCENY,3,09/06/2018 03:30:10 AM,"2300 TELEGRAPH AVE\nBerkeley, CA\n(37.868714, ...",2300 TELEGRAPH AVE,Berkeley,CA
3784,18040137,DISTURBANCE,2018-07-17,10:34:00,DISORDERLY CONDUCT,2,09/06/2018 03:30:13 AM,"1100 UNIVERSITY AVE\nBerkeley, CA\n(37.869067,...",1100 UNIVERSITY AVE,Berkeley,CA
3785,18090816,VANDALISM,2018-05-16,20:00:00,VANDALISM,3,09/06/2018 03:30:13 AM,"800 VICENTE RD\nBerkeley, CA\n",800 VICENTE RD,Berkeley,CA
3786,18024397,SEXUAL ASSAULT FEL.,2018-04-28,17:00:00,SEX CRIME,6,09/06/2018 03:30:12 AM,"2700 BANCROFT WAY\nBerkeley, CA\n(37.869312, -...",2700 BANCROFT WAY,Berkeley,CA


Now we can use the `dt` accessor on this column.

We can get the month:

In [19]:
# get months from EVENTDT
calls["EVENTDT"].dt.month

0       4
1       5
2       5
3       3
4       3
       ..
3783    8
3784    7
3785    5
3786    4
3787    4
Name: EVENTDT, Length: 3788, dtype: int32

Which day of the week the date is on:

In [20]:
# get week days from EVENTDT
calls["EVENTDT"].dt.dayofweek

0       2
1       2
2       4
3       1
4       5
       ..
3783    2
3784    1
3785    2
3786    5
3787    3
Name: EVENTDT, Length: 3788, dtype: int32

Check the mimimum values to see if there are any suspicious-looking, 70s dates:

In [ ]:
# Answer here

,CASENO,OFFENSE,EVENTDT,EVENTTM,CVLEGEND,CVDOW,InDbDate,Block_Location,BLKADDR,City,State
2513,20057398,BURGLARY COMMERCIAL,2020-12-17,16:05,BURGLARY - COMMERCIAL,4,06/15/2021 12:00:00 AM,"600 BLOCK GILMAN ST\nBerkeley, CA\n(37.878405,...",600 BLOCK GILMAN ST,Berkeley,CA
624,20057207,ASSAULT/BATTERY MISD.,2020-12-17,16:50,ASSAULT,4,06/15/2021 12:00:00 AM,"2100 BLOCK SHATTUCK AVE\nBerkeley, CA\n(37.871...",2100 BLOCK SHATTUCK AVE,Berkeley,CA
154,20092214,THEFT FROM AUTO,2020-12-17,18:30,LARCENY - FROM VEHICLE,4,06/15/2021 12:00:00 AM,"800 BLOCK SHATTUCK AVE\nBerkeley, CA\n(37.8918...",800 BLOCK SHATTUCK AVE,Berkeley,CA
659,20057324,THEFT MISD. (UNDER $950),2020-12-17,15:44,LARCENY,4,06/15/2021 12:00:00 AM,"1800 BLOCK 4TH ST\nBerkeley, CA\n(37.869888, -...",1800 BLOCK 4TH ST,Berkeley,CA
993,20057573,BURGLARY RESIDENTIAL,2020-12-17,22:15,BURGLARY - RESIDENTIAL,4,06/15/2021 12:00:00 AM,"1700 BLOCK STUART ST\nBerkeley, CA\n(37.857495...",1700 BLOCK STUART ST,Berkeley,CA


Doesn't look like it! We are good!


We can also do many things with the `dt` accessor like switching time zones and converting time back to UNIX/POSIX time. Check out the documentation on [`.dt` accessor](https://pandas.pydata.org/docs/user_guide/basics.html#basics-dt-accessors) and [time series/date functionality](https://pandas.pydata.org/docs/user_guide/timeseries.html#).


## Data Faithfulness: Mauna Loa CO2 data

CO2 concentrations have been monitored at Mauna Loa Observatory since 1958 ([website link](https://gml.noaa.gov/ccgg/trends/data.html)).




In [21]:
co2_file = "co2_mm_mlo.txt"

Let's do some **EDA**!!

### How do we read the file into Pandas?
Let's instead check out this file with JupyterLab.

* Note it's a `.txt` file.
* Do we trust this file extension?
* What structure is it?


Looking at the first few lines of the data, we spot some relevant characteristics:

- The values are separated by white space, possibly tabs.
- The data line up down the rows. For example, the month appears in 7th to 8th position of each line.
- The 71st and 72nd lines in the file contain column headings split over two lines.

We can use `read_csv` to read the data into a Pandas data frame, and we provide several arguments to specify that the separators are white space, there is no header (**we will set our own column names**), and to skip the first 72 rows of the file.

In [23]:
# use pd.read_csv to read txt file
co2=pd.read_csv("co2_mm_mlo.txt", delim_whitespace=True, header=None,  skiprows=72)
co2

,0,1,2,3,4,5,6
0,1958,3,1958.21,315.71,315.71,314.62,-1
1,1958,4,1958.29,317.45,317.45,315.29,-1
2,1958,5,1958.38,317.50,317.50,314.71,-1
3,1958,6,1958.46,-99.99,317.10,314.85,-1
4,1958,7,1958.54,315.86,315.86,314.98,-1
...,...,...,...,...,...,...,...
733,2019,4,2019.29,413.32,413.32,410.49,26
734,2019,5,2019.38,414.66,414.66,411.20,28
735,2019,6,2019.46,413.92,413.92,411.58,27
736,2019,7,2019.54,411.77,411.77,411.43,23


Congratulations! You've wrangled the data!

<br/>

...But our columns aren't named.
**We need to do more EDA.**

### Exploring Variable Feature Types

The NOAA [webpage](https://gml.noaa.gov/ccgg/trends/) might have some useful tidbits (in this case it doesn't).
Let's go back to the raw data file to identify each feature.


We'll rerun `pd.read_csv`, but this time with some **custom column names.**

In [25]:
co2.describe()

,0,1,2,3,4,5,6
count,738.00,738.00,738.00,738.00,738.00,738.00,738.00
mean,1988.42,6.49,1988.92,350.47,354.50,354.48,18.47
std,17.77,3.44,17.77,52.21,28.11,28.03,12.20
min,1958.00,1.00,1958.21,-99.99,312.66,314.62,-1.00
25%,1973.00,4.00,1973.56,328.59,328.79,329.73,-1.00
50%,1988.00,6.00,1988.92,351.73,351.73,352.38,25.00
75%,2004.00,9.00,2004.27,377.00,377.00,377.18,28.00
max,2019.00,12.00,2019.62,414.66,414.66,411.84,31.00


Yikes! Plotting the data uncovered a problem. It looks like we have some **missing values**. What happened here?

In [26]:
co2.head()

,0,1,2,3,4,5,6
0,1958,3,1958.21,315.71,315.71,314.62,-1
1,1958,4,1958.29,317.45,317.45,315.29,-1
2,1958,5,1958.38,317.50,317.50,314.71,-1
3,1958,6,1958.46,-99.99,317.10,314.85,-1
4,1958,7,1958.54,315.86,315.86,314.98,-1


In [27]:
co2.tail()

,0,1,2,3,4,5,6
733,2019,4,2019.29,413.32,413.32,410.49,26
734,2019,5,2019.38,414.66,414.66,411.20,28
735,2019,6,2019.46,413.92,413.92,411.58,27
736,2019,7,2019.54,411.77,411.77,411.43,23
737,2019,8,2019.62,409.95,409.95,411.84,29
